In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from includes import utils
from includes import config
from includes import algorithms

In [ ]:
def kfold_evaluate_algorithm(dataset, algorithm, eval_seed, n_folds, verbose, *args):
    np.random.seed(eval_seed)
    folds = utils.folds(dataset, n_folds)
    np.random.seed(np.random.randint(0, 100))

    scores = list()
    for n_fold in range(n_folds):
        if verbose:
            print "\033[1mStarting fold %d\033[0m\n" % (n_fold + 1)
            
        indices = [True] * n_folds
        indices[n_fold] = False

        train_set = np.concatenate(folds[indices], axis=0)

        test_set = folds[n_fold]

        predicted = algorithm(train_set, test_set, verbose, *args)

        accuracy = np.mean(test_set[:, -1] == predicted) * 100
        scores.append(accuracy)

    return scores

In [ ]:
def evaluate_algorithm(train_set, test_set, algorithm, verbose, *args):
    np.random.seed(np.random.randint(0, 100))

    predicted, model = algorithm(train_set, test_set, verbose, *args)

    accuracy = np.mean(test_set[:, -1] == predicted) * 100

    return accuracy, model

## Diabetes Dataset

In [ ]:
dataset, _ = utils.get_data("diabetes")
conf = config.Config()

In [ ]:
conf.set_algorithm("log-reg")
scores = kfold_evaluate_algorithm(
    dataset,
    algorithms.logistic_regression,
    conf.eval_seed,
    conf.n_folds,
    conf.verbose,
    conf.lr,
    conf.n_epochs,
    conf.batch_size,
    conf.tol
)

print "Logistic Regression"
print "\tScores: %s" % scores
print "\tMean Accuracy: %.3f%%" % (sum(scores)/float(len(scores)))

In [ ]:
conf.set_algorithm("boosted-log-reg")
scores = kfold_evaluate_algorithm(
    dataset,
    algorithms.boosted_logistic_regression,
    conf.eval_seed,
    conf.n_folds,
    conf.verbose,
    conf.l_rate,
    conf.eta,
    conf.eta_decay,
    conf.n_epochs,
    conf.batch_size,
    conf.tol,
    conf.m_stop
)

print "Boosted Logistic Regression"
print "\tScores: %s" % scores
print "\tMean Accuracy: %.3f%%" % (sum(scores)/float(len(scores)))

In [ ]:
conf.set_algorithm("mlp")
scores = kfold_evaluate_algorithm(
    dataset,
    algorithms.mlp,
    conf.eval_seed,
    conf.n_folds,
    conf.verbose,
    conf.lr,
    conf.n_epochs,
    conf.batch_size,
    conf.hidden_size
)

print "Multi Layer Perceptron"
print "\tScores: %s" % scores
print "\tMean Accuracy: %.3f%%" % (sum(scores)/float(len(scores)))
print ""

## MNIST Dataset

In [ ]:
train_set, test_set, n_classes = utils.get_data("mnist")

In [ ]:
conf = config.Config()

In [ ]:
conf.set_algorithm("sft-reg")
accuracy = evaluate_algorithm(
    train_set,
    test_set,
    algorithms.softmax_regression,
    conf.verbose,
    conf.lr,
    n_classes,
    conf.n_epochs,
    conf.batch_size,
    conf.tol
)

print "Softmax Regression"
print "Accuracy: %s" % accuracy

In [ ]:
conf.set_algorithm("boosted-sft-reg")
accuracy, model = evaluate_algorithm(
    train_set,
    test_set,
    algorithms.boosted_softmax_regression,
    conf.verbose,
    conf.lr,
    n_classes,
    conf.n_epochs,
    conf.batch_size,
    conf.m_stop,
    conf.eta
)

print "Boosted Softmax Regression"
print "Final Test Accuracy: %s" % accuracy

In [ ]:
conf.set_algorithm("multi-mlp")
accuracy = evaluate_algorithm(
    train_set,
    test_set,
    algorithms.multi_mlp,
    conf.verbose,
    conf.lr,
    n_classes,
    conf.n_epochs,
    conf.batch_size,
    conf.hidden_size,
    conf.tol
)

print "Multi Layer Perceptron"
print "Accuracy: %s" % accuracy